# Final Project

William Nayden and Quynh Chau

## Data Import and Packages

In [74]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, log_loss, confusion_matrix

In [55]:
df = pd.read_csv("final_project.csv")
df = df.dropna()
target = df['y']
df.drop(['y'],inplace=True, axis=1)

In [56]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 158392 entries, 0 to 159999
Data columns (total 50 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   x0      158392 non-null  float64
 1   x1      158392 non-null  float64
 2   x2      158392 non-null  float64
 3   x3      158392 non-null  float64
 4   x4      158392 non-null  float64
 5   x5      158392 non-null  float64
 6   x6      158392 non-null  float64
 7   x7      158392 non-null  float64
 8   x8      158392 non-null  float64
 9   x9      158392 non-null  float64
 10  x10     158392 non-null  float64
 11  x11     158392 non-null  float64
 12  x12     158392 non-null  float64
 13  x13     158392 non-null  float64
 14  x14     158392 non-null  float64
 15  x15     158392 non-null  float64
 16  x16     158392 non-null  float64
 17  x17     158392 non-null  float64
 18  x18     158392 non-null  float64
 19  x19     158392 non-null  float64
 20  x20     158392 non-null  float64
 21  x21     15

In [57]:
obj_df = df.select_dtypes(include=['object']).copy()
obj_df.head()

,x24,x29,x30,x32,x37
0,euorpe,July,tuesday,0.0%,$1313.96
1,asia,Aug,wednesday,-0.02%,$1962.78
2,asia,July,wednesday,-0.01%,$430.47
3,asia,July,wednesday,0.01%,$-2366.29
4,asia,July,tuesday,0.01%,$-620.66


In [58]:
def per(x):
    return float(x.strip('%'))

df['x32'] = df['x32'].apply(per)
print(df['x32'])

0         0.00
1        -0.02
2        -0.01
3         0.01
4         0.01
          ... 
159995    0.00
159996   -0.01
159997   -0.00
159998   -0.02
159999    0.02
Name: x32, Length: 158392, dtype: float64


In [59]:
def dol(x):
    return float(x.strip('$'))

df['x37'] = df['x37'].apply(dol)
print(df['x37'])

0         1313.96
1         1962.78
2          430.47
3        -2366.29
4         -620.66
           ...   
159995    -891.96
159996    1588.65
159997     687.46
159998     439.21
159999   -1229.34
Name: x37, Length: 158392, dtype: float64


In [65]:
from sklearn.preprocessing import LabelEncoder

df[['x24','x29','x30']] = df[['x24','x29','x30']].apply(LabelEncoder().fit_transform)
print(df[['x24','x29','x30']])

        x24  x29  x30
0         2    5    3
1         1    1    4
2         1    5    4
3         1    5    4
4         1    5    3
...     ...  ...  ...
159995    1    1    4
159996    1    8    4
159997    1    6    4
159998    1    8    4
159999    1    1    3

[158392 rows x 3 columns]


In [66]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 158392 entries, 0 to 159999
Data columns (total 50 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   x0      158392 non-null  float64
 1   x1      158392 non-null  float64
 2   x2      158392 non-null  float64
 3   x3      158392 non-null  float64
 4   x4      158392 non-null  float64
 5   x5      158392 non-null  float64
 6   x6      158392 non-null  float64
 7   x7      158392 non-null  float64
 8   x8      158392 non-null  float64
 9   x9      158392 non-null  float64
 10  x10     158392 non-null  float64
 11  x11     158392 non-null  float64
 12  x12     158392 non-null  float64
 13  x13     158392 non-null  float64
 14  x14     158392 non-null  float64
 15  x15     158392 non-null  float64
 16  x16     158392 non-null  float64
 17  x17     158392 non-null  float64
 18  x18     158392 non-null  float64
 19  x19     158392 non-null  float64
 20  x20     158392 non-null  float64
 21  x21     15

## XGBoost Model

In [67]:
# split data into train and test sets
seed = 7
test_size = 0.2
X_train, X_test, y_train, y_test = train_test_split(df, target, test_size=test_size, random_state=seed)

In [68]:
xgtrain = xgb.DMatrix(X_train.values, y_train.values)
xgtest = xgb.DMatrix(X_test.values, y_test.values)

In [69]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

In [70]:
# "Learn" the mean from the training data
mean_train = np.mean(y_train)
# Get predictions on the test set
baseline_predictions = np.ones(y_test.shape) * mean_train

# Compute Log Loss and Accuracy
log_baseline = log_loss(y_test, baseline_predictions)
acc_baseline = accuracy_score(y_test, np.ones(baseline_predictions.shape))

print("Baseline Accuracy is {:.2f}".format(acc_baseline))
print("Baseline Log Loss is {:.2f}".format(log_baseline))

Baseline Accuracy is 0.40
Baseline Log Loss is 0.67


In [71]:
# fit model no training data
model = xgb.XGBClassifier()
model.fit(X_train, y_train)

C:\Users\WilliamNayden\anaconda3\envs\DS7331\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[19:28:35] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=8, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [72]:
# make predictions for test data
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

In [73]:
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 92.43%


In [75]:
# evaluate predictions
cm = confusion_matrix(y_test, predictions)
print(cm)

[[17821  1047]
 [ 1352 11459]]


In [79]:
total_loss = (1047*225) + (1352*35)
formatted = "${:,.2f}".format(total_loss)
print(formatted)

$282,895.00


In [80]:
params = {
    # Parameters that we are going to tune.
    'max_depth':6,
    'min_child_weight': 1,
    'eta':.3,
    'subsample': 1,
    'colsample_bytree': 1,
    # Other parameters
    'objective':'binary:hinge',
}

In [81]:
params['eval_metric'] = "error"

In [83]:
num_boost_round = 999

model = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    evals=[(dtest, "Test")],
    early_stopping_rounds=10
)

[0]	Test-error:0.59560
[1]	Test-error:0.52628
[2]	Test-error:0.26383
[3]	Test-error:0.22052
[4]	Test-error:0.20559
[5]	Test-error:0.19499
[6]	Test-error:0.18549
[7]	Test-error:0.17071
[8]	Test-error:0.16304
[9]	Test-error:0.15717
[10]	Test-error:0.15357
[11]	Test-error:0.14991
[12]	Test-error:0.14262
[13]	Test-error:0.13801
[14]	Test-error:0.13548
[15]	Test-error:0.13372
[16]	Test-error:0.12983
[17]	Test-error:0.12904
[18]	Test-error:0.12545
[19]	Test-error:0.12431
[20]	Test-error:0.12210
[21]	Test-error:0.12011
[22]	Test-error:0.11992
[23]	Test-error:0.11916
[24]	Test-error:0.11784
[25]	Test-error:0.11326
[26]	Test-error:0.10922
[27]	Test-error:0.10673
[28]	Test-error:0.10398
[29]	Test-error:0.10275
[30]	Test-error:0.10127
[31]	Test-error:0.09994
[32]	Test-error:0.09782
[33]	Test-error:0.09754
[34]	Test-error:0.09666
[35]	Test-error:0.09514
[36]	Test-error:0.09454
[37]	Test-error:0.09385
[38]	Test-error:0.09281
[39]	Test-error:0.09211
[40]	Test-error:0.09082
[41]	Test-error:0.09022
[4

In [84]:
print("Best Accuracy: {:.2f} with {} rounds".format(
                 (1-model.best_score),
                 model.best_iteration+1))

Best Accuracy: 0.93 with 129 rounds


In [85]:
cv_results = xgb.cv(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    seed=42,
    nfold=5,
    metrics={'logloss','error'},
    early_stopping_rounds=10
)
cv_results

,train-error-mean,train-error-std,train-logloss-mean,train-logloss-std,test-error-mean,test-error-std,test-logloss-mean,test-logloss-std
0,0.599607,0.001213,22.091349,0.044683,0.599607,0.004851,22.090093,0.178729
1,0.518818,0.023295,19.114787,0.858270,0.520349,0.019487,19.170089,0.717906
2,0.280171,0.007274,10.322113,0.268003,0.285006,0.006905,10.500096,0.254380
3,0.221025,0.003190,8.142972,0.117538,0.226812,0.003264,8.356120,0.120233
4,0.197040,0.002584,7.259264,0.095221,0.203278,0.003724,7.489102,0.137180
...,...,...,...,...,...,...,...,...
141,0.047523,0.002106,1.750815,0.077569,0.078840,0.002521,2.904559,0.092868
142,0.047454,0.002088,1.748271,0.076946,0.078737,0.002559,2.900779,0.094268
143,0.047397,0.002083,1.746163,0.076751,0.078713,0.002559,2.899907,0.094290
144,0.047343,0.002038,1.744200,0.075099,0.078642,0.002563,2.897290,0.094432


In [88]:
print('Best Accuracy: {}'.format(1-cv_results['test-error-mean'].min()))
print('Best Log Loss: {}'.format(cv_results['test-logloss-mean'].min()))

Best Accuracy: 0.9214132
Best Log Loss: 2.8952547999999996


In [89]:
gridsearch_params = [
    (max_depth, min_child_weight)
    for max_depth in range(9,12)
    for min_child_weight in range(5,8)
]

In [91]:
# Define initial best params and accuracy
min_error = float("Inf")
best_params = None
for max_depth, min_child_weight in gridsearch_params:
    print("CV with max_depth={}, min_child_weight={}".format(
                             max_depth,
                             min_child_weight))
    # Update our parameters
    params['max_depth'] = max_depth
    params['min_child_weight'] = min_child_weight
    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'logloss', 'error'},
        early_stopping_rounds=10
    )
    # Update best Log Loss
    mean_logloss = cv_results['test-logloss-mean'].min()
    mean_error = cv_results['test-error-mean'].min()
    boost_rounds = cv_results['test-error-mean'].argmin()
    print("\Log Loss {} for {} rounds".format(mean_logloss, boost_rounds))
    print("\Accuracy {} for {} rounds".format((1-mean_error), boost_rounds))
    if mean_error < min_error:
        min_error = mean_error
        best_params = (max_depth,min_child_weight)
print("Best params: {}, {}, Log Loss: {}, Accuracy: {}".format(best_params[0], best_params[1], mean_logloss, (1-min_error)))

CV with max_depth=9, min_child_weight=5
\Log Loss 2.8461178 for 39 rounds
\Accuracy 0.9227468 for 39 rounds
CV with max_depth=9, min_child_weight=6
\Log Loss 2.8484432 for 61 rounds
\Accuracy 0.9226836 for 61 rounds
CV with max_depth=9, min_child_weight=7
\Log Loss 2.7900036 for 49 rounds
\Accuracy 0.9242698 for 49 rounds
CV with max_depth=10, min_child_weight=5
\Log Loss 2.8443721999999996 for 32 rounds
\Accuracy 0.9227942 for 32 rounds
CV with max_depth=10, min_child_weight=6
\Log Loss 2.7905852 for 33 rounds
\Accuracy 0.924254 for 33 rounds
CV with max_depth=10, min_child_weight=7
\Log Loss 2.811227 for 32 rounds
\Accuracy 0.9236936 for 32 rounds
CV with max_depth=11, min_child_weight=5
\Log Loss 2.888276 for 33 rounds
\Accuracy 0.9216022 for 33 rounds
CV with max_depth=11, min_child_weight=6
\Log Loss 2.8350698000000003 for 24 rounds
\Accuracy 0.9230465999999999 for 24 rounds
CV with max_depth=11, min_child_weight=7
\Log Loss 2.8449542 for 25 rounds
\Accuracy 0.9227782 for 25 round

## KERAS Model

In [92]:
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.
C:\Users\WilliamNayden\anaconda3\envs\DS7331\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\WilliamNayden\anaconda3\envs\DS7331\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\WilliamNayden\anaconda3\envs\DS7331\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\WilliamNayd

In [93]:
# define the keras model
model = Sequential()
model.add(Dense(12, input_dim=50, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [94]:
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [95]:
# fit the keras model on the dataset
model.fit(df, target, epochs=150, batch_size=10)


Epoch 1/150
158392/158392 [==============================] - 13s 85us/step - loss: 0.5187 - accuracy: 0.77013s - loss: 0.5577 - accuracy - ETA:  - ETA: 2s - loss: 0.5436 - accuracy - ETA: 2s - loss: 0.5413 - accuracy - ETA: 1s - loss: 0.5
Epoch 2/150
158392/158392 [==============================] - 12s 76us/step - loss: 0.3289 - accuracy: 0.85970s - loss: 0.3296  - ETA: 0s - loss: 0.3289 - accuracy: 0.85
Epoch 3/150
158392/158392 [==============================] - 12s 76us/step - loss: 0.2846 - accuracy: 0.88199s - loss: 0.2968 - accura
Epoch 4/150
158392/158392 [==============================] - 12s 74us/step - loss: 0.2577 - accuracy: 0.8952
Epoch 5/150
158392/158392 [==============================] - 12s 77us/step - loss: 0.2422 - accuracy: 0.90236s - loss: 0.2459 - accuracy:  - - - ETA:  - ETA: 0s - loss: 0.2418 - accuracy: 0. - ETA: 0s - loss: 0.2421 - ac - ETA: 0s - loss: 0.2423 - accuracy: 
Epoch 6/150
158392/158392 [==============================] - 13s 82us/step - loss: 0.232

158392/158392 [==============================] - 13s 82us/step - loss: 0.1900 - accuracy: 0.9285: 13s - loss: 0.1861 - accurac - ETA: 12s  - ETA: 10s  - ETA: 9s - loss: 0.1953 - accuracy: 0. - ETA: 9s - los - ETA: 9s - l - ETA: 8s - - ETA: 6s - loss: 0.1916 - ac - ETA: 3s - loss: 0.1894 -  - ETA: 1s - loss: 0.1892 - accuracy - ETA: 1s - los - ETA: 0s - loss: 0 - ETA: 0s - loss: 0.1894 - ac
Epoch 27/150
158392/158392 [==============================] - ETA: 0s - loss: 0.1901 - accuracy: 0.9279 - ETA: 11s - loss: 0.1981 - accura - ETA: 10s - loss: 0.19 - 13s 81us/step - loss: 0.1903 - accuracy: 0.9279
Epoch 28/150
158392/158392 [==============================] - 13s 80us/step - loss: 0.1894 - accuracy: 0.9285: 11s - loss: 0.1891 - accuracy: 0. - ETA: 12s - loss: 0.1890 - acc - ETA: 11
Epoch 29/150
158392/158392 [==============================] - 13s 82us/step - loss: 0.1890 - accuracy: 0.92886s - l - - ETA: 4s - loss: 0.1902 - accuracy: 0. - ETA: 4s - l - ETA: 3s - loss: 0.189 - ETA: 1s -

158392/158392 [==============================] - 14s 87us/step - loss: 0.1815 - accuracy: 0.9332A: 13s - loss: 0.2019 - ac - ETA: 11s - loss: 0.1864 -  - ETA: 9s - loss: 0.1822 - accuracy:  - ETA: 9s - loss: 0.1816 - accuracy:  - ETA: 9s - loss: 0.1 - ETA:  - ETA: 3s - loss: 0.1806 -  - ETA: 3s - loss: - ETA: 2s - - ETA: 1s -
Epoch 49/150
158392/158392 [==============================] - 13s 83us/step - loss: 0.1804 - accuracy: 0.9338: 4s - loss: 0.1 - ETA: 3s - - ETA: 0s - loss: 0.180
Epoch 50/150
158392/158392 [==============================] - 13s 80us/step - loss: 0.1812 - accuracy: 0.9328: 11s - loss: 0.1845 - accuracy:  - ETA: 11s - los - ETA: 10s - loss: 0.1833 - accuracy: 0.932 - ETA: 10s - loss: 0 - ETA: 5s - loss: 0.1797 - ac - ETA: 5s - loss: 0.1798 - accuracy: 0.93 - ETA: 5s - loss: 0.1799 - accuracy: 0.93 - ETA: 5s - los - ETA: 1s - loss: 0.1 - ETA: 0s - loss: 0.1814 - accura
Epoch 51/150
158392/158392 [==============================] - 13s 81us/step - loss: 0.1797 - accura

158392/158392 [==============================] - 12s 77us/step - loss: 0.1733 - accuracy: 0.9376: 13s - loss: 0.1795 - accuracy: 0.934 - ETA: 12s - loss: 0.1738 - a - ETA: 11s - loss: 0.1725 - accur - ETA: 10s - loss: 0.1701 - accuracy - ETA: 10s - loss: 0.1722 - accuracy: 0.9 - ETA: 10 - ETA: 7s - loss: 0.1 - ETA: 2s - loss: 0
Epoch 78/150
158392/158392 [==============================] - 12s 74us/step - loss: 0.1735 - accuracy: 0.9376
Epoch 79/150
158392/158392 [==============================] - 13s 80us/step - loss: 0.1736 - accuracy: 0.93710s - loss: 0.1735 - accuracy:  - ETA: 0s - loss: 0.1
Epoch 80/150
158392/158392 [==============================] - 12s 74us/step - loss: 0.1732 - accuracy: 0.93753s - loss: 0.1738 - accuracy: 0.93 - ETA: 
Epoch 81/150
158392/158392 [==============================] - 12s 75us/step - loss: 0.1730 - accuracy: 0.93770s - loss: 0.1730 - accuracy: 0.93
Epoch 82/150
158392/158392 [==============================] - 12s 76us/step - loss: 0.1730 - accuracy:

158392/158392 [==============================] - 12s 77us/step - loss: 0.1708 - accuracy: 0.93919s - loss: 0.1 - ETA: 8s - ETA: 3s - loss: 0.1 - ETA: 1s - ETA: 0s -
Epoch 105/150
158392/158392 [==============================] - 12s 77us/step - loss: 0.1699 - accuracy: 0.93921s - ETA: 0s - loss: 0.1705 - accuracy: 0. - ETA: 0s - loss: 0
Epoch 106/150
158392/158392 [==============================] - 12s 77us/step - loss: 0.1707 - accuracy: 0.93993s - loss: 0.1716 - accuracy: 0.93 - ETA: 3s - loss: 0.1715 
Epoch 107/150
158392/158392 [==============================] - 12s 75us/step - loss: 0.1696 - accuracy: 0.93941s - loss: 0.1696 - accuracy: 0. - ETA: 1s - loss: 0.1696 -  - ETA: 0s - los - ETA: 0s - loss: 0.1694 - accuracy: 0.
Epoch 108/150
158392/158392 [==============================] - 13s 80us/step - loss: 0.1701 - accuracy: 0.9397
Epoch 109/150
158392/158392 [==============================] - 12s 74us/step - loss: 0.1698 - accuracy: 0.9400A: 14s - loss: 0
Epoch 110/150
158392/15839

158392/158392 [==============================] - 12s 77us/step - loss: 0.1685 - accuracy: 0.94077s - loss: 0.1633 - accuracy: 0.94 - ETA: 7s - loss: 0.1630 - accuracy: 0.94 - ETA: 7s - loss: 0.1630 - accuracy: 0. - ETA: 7s - loss: 0.1643 - ac - ETA: 0s - loss: 0.1684 - 
Epoch 132/150
158392/158392 [==============================] - 13s 81us/step - loss: 0.1681 - accuracy: 0.9399
Epoch 133/150
158392/158392 [==============================] - 12s 75us/step - loss: 0.1685 - accuracy: 0.94019s - los - ETA: 8s - loss: 0.1718 - accuracy:  - ETA: 8s - los -
Epoch 134/150
158392/158392 [==============================] - 13s 81us/step - loss: 0.1681 - accuracy: 0.94082s - - ETA: 1s - loss: 0.1
Epoch 135/150
158392/158392 [==============================] - 12s 79us/step - loss: 0.1677 - accuracy: 0.9407TA: 0s - loss: 0.1683 - accuracy: 0.94 - ETA: 0s - l
Epoch 136/150
158392/158392 [==============================] - ETA: 0s - loss: 0.1688 - accuracy: 0.9403 - ETA: 11s - loss: 0.1691 - accu - ETA

In [96]:
# evaluate the keras model
_, accuracy = model.evaluate(df, target)
print('Accuracy: %.2f' % (accuracy*100))

158392/158392 [==============================] - 2s 12us/step
Accuracy: 94.14


In [107]:
# make predictions for test data
y_pred = model.predict_classes(df)
predictions = y_pred.tolist()
#for value in y_pred:
    #round(value)
   # predictions.append(value)
    
print(predictions)

[[0], [0], [0], [0], [1], [0], [0], [0], [0], [1], [1], [0], [0], [1], [0], [0], [0], [0], [1], [1], [0], [0], [1], [0], [1], [0], [0], [0], [1], [0], [1], [0], [0], [0], [0], [0], [1], [1], [0], [0], [0], [0], [0], [1], [0], [0], [0], [1], [1], [0], [1], [1], [0], [0], [0], [1], [1], [1], [1], [1], [0], [0], [0], [0], [0], [0], [0], [0], [1], [1], [0], [1], [0], [0], [0], [0], [1], [1], [1], [1], [1], [1], [1], [1], [0], [1], [1], [0], [0], [1], [0], [1], [0], [0], [0], [0], [0], [0], [1], [0], [0], [0], [0], [1], [0], [0], [1], [1], [0], [0], [1], [0], [0], [0], [1], [0], [0], [0], [1], [0], [1], [1], [1], [0], [1], [1], [1], [0], [0], [0], [0], [1], [1], [0], [1], [1], [1], [1], [0], [1], [0], [1], [1], [0], [0], [0], [1], [0], [0], [1], [1], [0], [1], [1], [0], [0], [1], [0], [0], [1], [0], [1], [0], [1], [1], [1], [0], [0], [1], [1], [1], [0], [1], [0], [1], [1], [0], [1], [0], [0], [1], [0], [0], [0], [0], [1], [1], [1], [1], [0], [0], [1], [0], [1], [1], [0], [0], [0], [0], [0],

In [108]:
# evaluate predictions
cm = confusion_matrix(target, predictions)
print(cm)

[[90586  4260]
 [ 5014 58532]]
